In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Diabetes Prediction with Logistic Regression  
Veri seti ABD'deki Ulusal Diyabet-Sindirim-Böbrek Hastalıkları Enstitüleri'nde tutulan büyük veri setinin parçasıdır.

ABD'deki Arizona Eyaleti'nin en büyük 5. şehri olan Phoenix şehrinde yaşayan 21 yaş ve üzerinde olan Pima Indian kadınları üzerinde yapılan diyabet araştırması için kullanılan verilerdir.

**Değişkenler**

**Pregnancies** – Hamilelik sayısı

**Glucose** – Kandaki gukoz değeri
-Oral glikoz tolerans testinde 2 saatlik plazma glikoz konsantrasyonu

**SkinThickness** – Cilt Kalınlığı

**BMI** – Vücut Kitle Endeksi

**Insulin** – 2 saatlik serum insülini (mu U/ml)

**Blood Pressure** – Kan Basıncı (Küçük tansiyon) (mm Hg)

**DiabetesPedigreeFunction**: – Fonksiyon
-Oral glikoz tolerans testinde 2 saatlik plazma glikoz konsantrasyonu

**Age** – Yaş (yıl)

**Outcome** – Hastalığa sahip (1) ya da değil (0)



# Gerekli Import İşlemleri

In [ ]:
import pandas as pd
from datetime import date

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import missingno as msno
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, \
    roc_auc_score, confusion_matrix, classification_report, plot_roc_curve
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter(action = "ignore")


# Data

In [ ]:
def load():
    data = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")
    return data
df = load()

# 1) Keşifçi Veri Analizi (EDA)

In [ ]:
print("##################### Shape #####################")
print(df.shape)
print("##################### Types #####################")
print(df.dtypes)
print("##################### Head #####################")
print(df.head())
print("##################### Tail #####################")
print(df.tail())
print("##################### NA #####################")
print(df.isnull().sum())
print("##################### Describe ##############")
print(df.describe([0.10,0.25,0.50,0.75,0.90,0.95,0.99]).T)

Veri setinde her ne kadar NA değer gözlemlenmiyor olsa da değişkenlerin çoğunda sıfır değeri bol miktarda gözlemlenmektedir. "min" ve "quantile" değerlerini gözlemleyecek olursak; SkinThickness, BMI ve Insulin gibi değişkenlerde sıfır değerinin ağırlığını gözlemleyebiliriz. İnsan vücudu için imkansıza yakın bir durum olduğu için biz sıfır değerlerine ilerleyen süreçlerde NA muamelesi yapacağız 

In [ ]:
# Hedef değişken değerleri 
df["Outcome"].value_counts()

In [ ]:
print(100 * df["Outcome"].value_counts() / len(df))

In [ ]:
#Sayısal ve Aynı Zamanda KAtegorik olmayan Değişkenleri Yakalıyoruz
num_cols = [col for col in df.columns if df[col].dtypes != "O"]
num_but_cat = [col for col in df.columns if df[col].nunique() < 10 and df[col].dtypes != "O"]
num_cols = [col for col in num_cols if col not in num_but_cat]
num_cols

In [ ]:
# Sayısal Değişkenler için Histogram Grafikleri 
fig, ax = plt.subplots(4,2, figsize=(16,16))
sns.distplot(df.Age, bins = 20, ax=ax[0,0]) 
sns.distplot(df.Pregnancies, bins = 20, ax=ax[0,1]) 
sns.distplot(df.Glucose, bins = 20, ax=ax[1,0]) 
sns.distplot(df.BloodPressure, bins = 20, ax=ax[1,1]) 
sns.distplot(df.SkinThickness, bins = 20, ax=ax[2,0])
sns.distplot(df.Insulin, bins = 20, ax=ax[2,1])
sns.distplot(df.DiabetesPedigreeFunction, bins = 20, ax=ax[3,0]) 
sns.distplot(df.BMI, bins = 20, ax=ax[3,1]) 

In [ ]:
# Değişkenlerin Hedef Değişkenin 0 ve 1 olma Durumuna Göre Ortalamaları 
for col in num_cols:
    print(df.groupby("Outcome").agg({col: "mean"}), end = "\n\n")

In [ ]:
# Değişkenlerin Birbirleri Arası Korelasyonu
corr = df.corr()
sns.heatmap(corr)
plt.show()

# 2) Veri Ön İşleme (Data Preprocessing)

# 2.1) Eksik Değerler (Missing Values)

In [ ]:
# Eksik Değerler 
df.isnull().sum()

In [ ]:
# 0 değerlerinin NA değerlere dönüştürülmesi 
zero_columns = ["Glucose","BloodPressure","Insulin","BMI","SkinThickness"]
for col in zero_columns:
    df[col] = np.where(df[col] == 0, np.nan, df[col])

In [ ]:
# Eksik Değerler 
df.isnull().sum()

In [ ]:
msno.matrix(df)

plt.show()

In [ ]:
# Oluşturduğumuz NA değerlere meadian ataması yapıyoruz. 
for col in num_cols:
    df[(df["Outcome"] == 0) & (df[col].isnull() == True)] = df[(df["Outcome"] == 0) & (df[col].isnull() == True)].fillna(df[df["Outcome"] == 0].median())
    df[(df["Outcome"] == 1) & (df[col].isnull() == True)] = df[(df["Outcome"] == 1) & (df[col].isnull() == True)].fillna(df[df["Outcome"] == 1].median())


In [ ]:
# Eksik Değerler ve Betimsel İstatistikler 
df.isnull().sum()


In [ ]:
df.describe().T

# 2.2) Aykırı Değerler (Outliers)


In [ ]:

for col in num_cols:   
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    up_limit = q3 + 1.5 * iqr
    low_limit = q1 - 1.5 * iqr
    if df[df[col] > up_limit].any(axis=None):
        print(col,"YES",(low_limit,up_limit))
    elif df[df[col] < low_limit].any(axis=None):
        print(col,"YES",(low_limit,up_limit))
    else: 
        print(col,"NO",(low_limit,up_limit))
        

In [ ]:
# Replace with Thresholds
for col in num_cols:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    up_limit = q3 + 1.5 * iqr
    low_limit = q1 - 1.5 * iqr
    df.loc[(df[col] < low_limit), col] = low_limit
    df.loc[(df[col] > up_limit), col] = up_limit
    if df[df[col] > up_limit].any(axis=None):
        print(col,"YES",(low_limit,up_limit))
    elif df[df[col] < low_limit].any(axis=None):
        print(col,"YES",(low_limit,up_limit))
    else: 
        print(col,"NO",(low_limit,up_limit))

In [ ]:
df.describe().T

# 3) Özellik Mühendisliği(Feature Engineering)

In [ ]:
def fea_eng(df):
    df["INPEDIGREE"] = df["Insulin"] * df["DiabetesPedigreeFunction"]
    df["SKINBMI"] = df["SkinThickness"] * df["BMI"]

    df.loc[(df["BloodPressure"] > 60) & (df["BloodPressure"] <= 80),"NEW_BP_CAT"]= "Ideal"
    df.loc[(df["BloodPressure"] > 80) & (df["BloodPressure"] < 90),"NEW_BP_CAT"]= "NormalBP"
    df.loc[(df["BloodPressure"] >= 90) & (df["BloodPressure"] < 120),"NEW_BP_CAT"]= "Hyper"
    df.loc[(df["BloodPressure"] >= 120),"NEW_BP_CAT"]= "Hypertensive crisis"
    df.loc[(df["BloodPressure"] <= 60),"NEW_BP_CAT"]= "Hypo"

    df.loc[(df["Pregnancies"] == 0) ,"NEW_P_CAT"]= "No_Child"
    df.loc[(df["Pregnancies"] > 0) & (df["Pregnancies"] <= 3 ),"NEW_P_CAT"]= "Child_03"
    df.loc[(df["Pregnancies"] > 3) & (df["Pregnancies"] <= 6 ),"NEW_P_CAT"]= "Child_36"
    df.loc[(df["Pregnancies"] > 6) ,"NEW_P_CAT"]= "Too_much"

    df.loc[(df["BMI"] < 18.5) ,"NEW_BMI_CAT"]= "UnderWeight"
    df.loc[(df["BMI"] >= 18.5) & (df["BMI"] <= 24.9 ),"NEW_BMI_CAT"]= "NormalBMI"
    df.loc[(df["BMI"] >= 25) & (df["BMI"] <= 30 ),"NEW_BMI_CAT"]= "Overweight"
    df.loc[(df["BMI"] > 30) & (df["BMI"] <= 35 ),"NEW_BMI_CAT"]= "Type1_Obese"
    df.loc[(df["BMI"] > 35) & (df["BMI"] <= 40 ),"NEW_BMI_CAT"]= "Type2_Obese"
    df.loc[(df["BMI"] > 40) ,"NEW_BMI_CAT"]= "Morbid_Obese"

    df.loc[(df["Insulin"] < 70) ,"NEW_I_CAT"]= "Hypoglycemia"
    df.loc[(df["Insulin"] >= 70) & (df["Insulin"] < 100 ),"NEW_I_CAT"]= "NormalI"
    df.loc[(df["Insulin"] >= 100) & (df["Insulin"] <= 125 ),"NEW_I_CAT"]= "PrediabetityI"
    df.loc[(df["Insulin"] >= 126) ,"NEW_I_CAT"]= "DiabetesI"

    df.loc[(df["Glucose"] < 100) ,"NEW_G_CAT"]= "Lower"
    df.loc[(df["Glucose"] >= 100) & (df["Glucose"] < 140 ),"NEW_G_CAT"]= "NormalG"
    df.loc[(df["Glucose"] >= 140) & (df["Glucose"] <= 199 ),"NEW_G_CAT"]= "PrediabetityG"
    df.loc[(df["Glucose"] >= 200) ,"NEW_G_CAT"]= "DiabetesG"

    df.loc[(df["Age"] < 30) ,"NEW_AGE_CAT"]= "Young"
    df.loc[(df["Age"] >= 30) & (df["Age"] < 40 ),"NEW_AGE_CAT"]= "Middle"
    df.loc[(df["Age"] >= 40) & (df["Age"] < 50 ),"NEW_AGE_CAT"]= "Old"
    df.loc[(df["Age"] >= 50) ,"NEW_AGE_CAT"]= "Elder"
    return df

In [ ]:
df = fea_eng(df)

In [ ]:
df.columns = [col.upper() for col in df.columns]

In [ ]:
# Feature Engineering sonucunda oluşan eksik değer var mı? Kontrol edelim.
df.isnull().sum()

In [ ]:
# Feature Engineering Sonrası Değişken Tutma İşlemini Tekrarlamalıyız
num_cols = [col for col in df.columns if df[col].dtypes != "O"]
num_but_cat = [col for col in df.columns if df[col].nunique() < 10 and df[col].dtypes != "O"]
num_cols = [col for col in num_cols if col not in num_but_cat]
num_cols

In [ ]:
# Kategorik Değişkenler Oluşturduğumuz için Onları da Yakalıyoruz.
cat_cols = [col for col in df.columns if df[col].dtypes == "O"]
cat_cols = cat_cols + num_but_cat
cat_cols

# 4) Encoding and Scale

# 4.1) Label Encoding

In [ ]:
binary_col = [col for col in df.columns if
                  df[col].dtype not in [int, float] and df[col].nunique() == 2]
for col in binary_col:
    labelencoder = LabelEncoder()
    df[col] = labelencoder.fit_transform(df[col])

# 4.2) One-Hot Encoding

In [ ]:
ohe_cols = [col for col in df.columns if 10 >= df[col].nunique() > 2]
df = pd.get_dummies(df, columns=ohe_cols, drop_first = True)

# 4.3) Robust Scaler

In [ ]:
for col in num_cols:
    transformer = RobustScaler().fit(df[[col]])
    df[col] = transformer.transform(df[[col]])

# 5) Model ve Final 

In [ ]:
y = df["OUTCOME"]
X = df.drop(["OUTCOME"], axis=1)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.20, random_state=1)

In [ ]:
log_model = LogisticRegression().fit(X_train, y_train)
log_model.intercept_


In [ ]:
log_model.coef_

In [ ]:
# Tahmin'lerin oluşturulması ve kaydedilmesi
y_pred = log_model.predict(X_train)
y_pred[0:10]

In [ ]:
y_train[0:10]

In [ ]:
# Sınıf olasılıkları (0. indextekiler 0 a ait olma 1. indekstekiler 1 sınıfına ait olma)
log_model.predict_proba(X_train)[0:10]

In [ ]:
# 1. sınıfa ait olma olasılıkları:
y_prob = log_model.predict_proba(X_train)[:, 1]

# Final Başarı Değelendirme 

In [ ]:
# Train Accuracy
y_pred = log_model.predict(X_train)
accuracy_score(y_train, y_pred)

In [ ]:
# Test
# AUC Score için y_prob
y_prob = log_model.predict_proba(X_test)[:, 1]

In [ ]:
# Diğer metrikler için y_pred
y_pred = log_model.predict(X_test)

In [ ]:
# ACCURACY
accuracy_score(y_test, y_pred)   

In [ ]:
# PRECISION
precision_score(y_test, y_pred)

In [ ]:
# RECALL
recall_score(y_test, y_pred)

In [ ]:
# F1
f1_score(y_test, y_pred)

In [ ]:
# Roc Eğrisi
plot_roc_curve(log_model, X_test, y_test)
plt.title('ROC Curve')
plt.plot([0, 1], [0, 1], 'r--')
plt.show()

In [ ]:
# AUC
roc_auc_score(y_test, y_prob)

In [ ]:
# Modeli oluşturduğumuz değişkenlerin etki düzeylerini gösteren feature importance grafiğini oluşturuyoruz.
import math as mt
feature_importance = pd.DataFrame(X_train.columns, columns = ["feature"])
feature_importance["importance"] = pow(mt.e, log_model.coef_[0])
feature_importance = feature_importance.sort_values(by = ["importance"], ascending=False)
ax = feature_importance.plot.barh(x='feature', y='importance', figsize=(15,10), fontsize=10)
plt.xlabel('Importance', fontsize=20)
plt.ylabel('Features', fontsize=10)
# plt.yticks(rotation=50)
plt.show()

Umarım faydalı olmuştur. Konu ile ilgili yazdığım medium yazısına aşağıdaki linkten ulaşabilirsiniz. 

https://kutlucagdas.medium.com/ad%C4%B1m-ad%C4%B1m-%C3%B6zellik-m%C3%BChendisli%C4%9Fi-8cf4fa0e5f81